In [5]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [6]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
nome_compat = 'PIC1022-SSSQQQ-MSP'
nome_A = 'PIC1022'
nome_B = 'SSSQQQ'

## Importar dados de compatibilidade

In [10]:
# Importar perímetros compatíveis
gdf_perim = gpd.read_file(f'{nome_compat}/perimetros_compativeis.gpkg')[['CD_PICSQ','geometry']]

# Importar matrizes de compatibilidade
matriz_A = pd.read_csv(f'{nome_compat}/matriz_compat_{nome_A}.csv',
                       sep='\t',
                       dtype={'CD_PERIMETRO':str, 'CD_PICSQ':str})
matriz_B = pd.read_csv(f'{nome_compat}/matriz_compat_{nome_B}.csv',
                       sep='\t',
                       dtype={'SSSQQQ':str, 'CD_PICSQ':str})

In [11]:
# Importar dados de PC já agregados
gdf_data = gpd.read_file('2010-2022-RMSP-RMBS-AUJ/perimetros_compativeis_agg.gpkg')

## Reagregação

In [12]:
# Reagregação
cols = ['dom_col_2022', 'dom_par_2022', 'dom_pi_2010',
       'dom_po_2010', 'dom_po_2022', 'dom_po_imput_2022', 'dom_pp_2010',
       'dom_tot_2010', 'dom_tot_2022', 'pop_dom_po_2022', 'pop_dom_pp_2010',
       'pop_tot_2010', 'pop_tot_2022']
agg = gdf_data.merge(matriz_A, on='CD_PERIMETRO', how='right')
agg = agg.pivot_table(index='CD_PICSQ', values=cols, aggfunc='sum').reset_index()

In [13]:
gdf_perim_agg = gdf_perim.merge(agg, on='CD_PICSQ', how='left')

# Variação de população
gdf_perim_agg['var_pop_tot'] = (gdf_perim_agg['pop_tot_2022']-gdf_perim_agg['pop_tot_2010'])

# Variação de domicílios
gdf_perim_agg['var_dom_tot'] = (gdf_perim_agg['dom_tot_2022']-gdf_perim_agg['dom_tot_2010'])

# Variação de domicílios
gdf_perim_agg['var_dom_po'] = (gdf_perim_agg['dom_po_2022']-gdf_perim_agg['dom_po_2010'])

# Variação de densidade populacional
gdf_perim_agg['var_dens_pop_tot_ha'] = gdf_perim_agg['var_pop_tot']/(gdf_perim_agg['geometry'].area/10000)

# Variação de densidade domiciliar (total)
gdf_perim_agg['var_dens_dom_tot_ha'] = gdf_perim_agg['var_dom_tot']/(gdf_perim_agg['geometry'].area/10000)

# Variação de densidade domiciliar (ocupados)
gdf_perim_agg['var_dens_dom_po_ha'] = gdf_perim_agg['var_dom_po']/(gdf_perim_agg['geometry'].area/10000)

# Variação da média de moradores por domicílio
gdf_perim_agg['med_pop_dom_po_2022'] = gdf_perim_agg['pop_dom_po_2022']/gdf_perim_agg['dom_po_2022']
gdf_perim_agg['var_med_pop_dom'] = gdf_perim_agg['med_pop_dom_po_2022']-(gdf_perim_agg['pop_dom_pp_2010']/gdf_perim_agg['dom_pp_2010'])

## Exportação

In [14]:
agg.to_csv(f'{nome_compat}/agg_perimetros_{nome_A}-{nome_B}.csv', sep='\t', index=False, decimal=',')

In [15]:
gdf_perim_agg.to_file(f'{nome_compat}/perimetros_compativeis_agg.gpkg',
                      layer=f'{nome_A}-{nome_B}',
                      driver='GPKG')